In [ ]:


#Ana Patricia Aparicio Reyes
# Funcionalidad: RF01 - Simulación de datos clínicos realistas


import pandas as pd
import numpy as np
from datetime import datetime, timedelta

# inicial dias configuración lol 

np.random.seed(42)
dias = 180
fecha_inicio = datetime(2025, 1, 1)
fechas = [fecha_inicio + timedelta(days=i) for i in range(dias)]

#  clima de panamá (poke solo es una persona de panamá pa que sea realista)

def clima_panama(fecha):
    mes = fecha.month
    if mes in [12, 1, 2, 3]:
        return np.random.choice(['Soleado', 'Seco', 'Ventoso'], p=[0.6, 0.3, 0.1])
    elif mes == 4:
        return np.random.choice(['Soleado', 'Seco', 'Ventoso'], p=[0.5, 0.3, 0.2])
    elif mes in [5, 11]:
        return np.random.choice(['Soleado', 'Nublado', 'Lluvioso', 'Húmedo'], p=[0.3, 0.3, 0.2, 0.2])
    else:
        return np.random.choice(['Lluvioso', 'Nublado', 'Húmedo'], p=[0.5, 0.3, 0.2])

clima = [clima_panama(f) for f in fechas]

# Temperatura 
temperatura = [np.random.uniform(26, 32) for _ in range(dias)]


#  estilo de vida( aqui ya se congigura lo del sueño, alimentación, brotes para que sea realista)

actividad_fisica = []
for f in fechas:
    dia_semana = f.weekday() 
    if dia_semana < 5:  
        act = np.random.choice([0, 1], p=[0.3, 0.7])
    else:  # fines de semana
        act = np.random.choice([0, 1], p=[0.6, 0.4])
    actividad_fisica.append(act)


# Sueño :  patrón de sueño diario
sueno_categorias = ['Horrible', 'Malo', 'Ok', 'Bueno', 'Excelente']
sueno_numerico = [np.random.choice([2,3,4])]
for i in range(1, dias):
    ayer = sueno_numerico[-1]
    # considerando que la calidad del sueño de hoy suele ser similar a la de ayer 
    hoy = np.clip(ayer + np.random.choice([-1,0,1], p=[0.2,0.6,0.2]), 1, 5)
    sueno_numerico.append(hoy)
sueno = [sueno_categorias[s-1] for s in sueno_numerico]

#  nutrición

categorias = ['Inflamatoria', 'Balanceada', 'Antiinflamatoria']
dieta_tipo = [np.random.choice(categorias) for _ in range(dias)]


# brotes de la enfermedad

n_brotes = np.random.randint(3,6)
brotes = np.zeros(dias)
for _ in range(n_brotes):
    inicio = np.random.randint(0, dias-14)
    duracion = np.random.randint(3,15)
    fin = min(inicio+duracion, dias)
    for i in range(inicio, fin):
        pos = (i-inicio)/duracion
        if pos < 0.3:
            brotes[i] = pos/0.3*2
        elif pos < 0.7:
            brotes[i] = 2 + np.random.uniform(-0.3,0.3)
        else:
            brotes[i] = 2*(1-(pos-0.7)/0.3)

# Convertimos dieta y clima a números
dieta_numerico = [categorias.index(d) for d in dieta_tipo]
clima_adverso = [1 if c in ['Lluvioso','Húmedo'] else 0 for c in clima]

# Listas para los síntomas
dolor_score = []
rigidez_score = []
inflamacion_score = []
fatiga_score = []

for i in range(dias):
    # Dolor
    dolor = 0.5 + brotes[i]                    # base por brote
    dolor += clima_adverso[i] * np.random.uniform(0.3,0.8)  # clima malo frio y así aumenta dolor
    dolor += (dieta_numerico[i]-1) * 0.3      # dieta antiinflamatoria reduce dolor
    if actividad_fisica[i]:
        dolor -= np.random.uniform(0,0.3)     # ejercicio reduce dolor
    dolor += (6 - sueno_numerico[i]) * 0.25   # poco sueño aumenta dolor o fatigs
    if i > 0:
        dolor += dolor_score[-1] * 0.3        # efecto del dolor del día anterior
    dolor += np.random.normal(0,0.2)          
    dolor = np.clip(round(dolor), 0, 5)
    dolor_score.append(dolor)

    #Rigidez 
    rigidez = round(dolor * np.random.uniform(0.7,1.0) + brotes[i]*0.4 + clima_adverso[i]*np.random.uniform(0.5,1.0))
    rigidez = np.clip(rigidez, 0, 5)
    rigidez_score.append(rigidez)

    #Inflamación
    inflamacion = round(dolor*0.6 + (dieta_numerico[i]-1)*0.4 + brotes[i]*0.5 + np.random.normal(0,0.3))
    inflamacion = np.clip(inflamacion, 0, 5)
    inflamacion_score.append(inflamacion)

    #Fatiga
    fatiga = round(dolor*0.4 + (5-sueno_numerico[i])*0.7 + brotes[i]*0.3 + clima_adverso[i]*0.3)
    if actividad_fisica[i]:
        fatiga -= 0.2  # ejercicio reduce fatiga
    fatiga = np.clip(round(fatiga + np.random.normal(0,0.2)), 0, 5)
    fatiga_score.append(fatiga)
    
# Estado emocional
estado_animo_score = []
estado_animo = []

for i in range(dias):
    score = 5 - dolor_score[i]*0.5 - fatiga_score[i]*0.3
    score += actividad_fisica[i]*0.4 + (sueno_numerico[i]-3)*0.2
    score += np.random.normal(0,0.4)

    # Limitar estado de ánimo 
    score = round(score)
    if score < 1:
        score = 1
    elif score > 5:
        score = 5

    estado_animo_score.append(score)

    # Categoría de ánimo
    if score <= 2:
        estado_animo.append(np.random.choice(['Triste','Frustrado','Irritable']))
    elif score == 3:
        estado_animo.append(np.random.choice(['Estresado','Ansioso','Irritable']))
    else:
        estado_animo.append(np.random.choice(['Tranquilo','Optimista','Feliz']))


#  dataframe

df = pd.DataFrame({
    'fecha': fechas,
    'clima': clima,
    'temperatura_C': np.round(temperatura,1),
    'actividad_fisica': actividad_fisica,
    'sueño': sueno,
    'dieta_tipo': dieta_tipo,
    'en_brote': (brotes>0.5).astype(int),
    'rigidez_score': rigidez_score,
    'dolor_score': dolor_score,
    'inflamacion_score': inflamacion_score,
    'fatiga_score': fatiga_score,
    'estado_animo': estado_animo,
    'estado_animo_score': estado_animo_score
})

print(df.head())

# Guardar el DataFrame en un CSV
df.to_csv("dataset_clinico_180dias.csv", index=False)



       fecha    clima  temperatura_C  actividad_fisica      sueño  \
0 2025-01-01  Soleado           28.0                 1      Bueno   
1 2025-01-02  Ventoso           26.7                 1      Bueno   
2 2025-01-03     Seco           31.5                 1      Bueno   
3 2025-01-04  Soleado           31.3                 0  Excelente   
4 2025-01-05  Soleado           27.5                 1  Excelente   

         dieta_tipo  en_brote  rigidez_score  dolor_score  inflamacion_score  \
0  Antiinflamatoria         0              1            1                  1   
1      Inflamatoria         1              2            2                  1   
2        Balanceada         1              3            3                  3   
3  Antiinflamatoria         1              3            3                  3   
4  Antiinflamatoria         1              5            4                  4   

   fatiga_score estado_animo  estado_animo_score  
0             1    Optimista                   5  
1 